# PERMANOVA analysis for recurrent mutation profiles

In [1]:
# Housekeeping

library(ggplot2)
library(reshape2)
library(RVAideMemoire)
library(scales)
library(vegan)

*** Package RVAideMemoire v 0.9-72 ***
Loading required package: permute
Loading required package: lattice
This is vegan 2.5-4


In [2]:
# Read in data

variants = read.table("../../../data/deep_seq/downsampled_multihit_nonsynonymous_variant_data.txt", 
                      sep = "\t", 
                      header = T)

In [3]:
variants2 = dcast(variants, TIME + ANTIBIOTIC + IMMIGRATION + REPLICATE + SPECIES ~ GENE, value.var = "COUNT")
head(variants2)

Aggregation function missing: defaulting to length


TIME,ANTIBIOTIC,IMMIGRATION,REPLICATE,SPECIES,ABNHBPOL_00951,ABNHBPOL_01122,ABNHBPOL_04066,ABNHBPOL_04069,AMIEGGJA_01338,⋯,rhsB_1,rsmG,slmA,spoT_1,tar_2,vgrG1_2,xcpW,xynD_5,ycdT_1,ydcV_4
8,0,0,1,HAMBI_1287,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,0,0,1,HAMBI_1875,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,0,0,1,HAMBI_1896,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,0,0,1,HAMBI_1972,1,1,2,0,0,⋯,0,0,0,0,0,3,1,0,0,0
8,0,0,1,HAMBI_1977,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,0,0,2,HAMBI_1287,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [4]:
rownames(variants2) = paste(variants2$TIME, 
                            variants2$ANTIBIOTIC,
                            variants2$IMMIGRATION, 
                            variants2$REPLICATE, 
                            variants2$SPECIES,
                            sep = "_")
meta = variants2[,1:5]
data = as.matrix(variants2[,-c(1:5)])
head(meta)
head(data)

,TIME,ANTIBIOTIC,IMMIGRATION,REPLICATE,SPECIES
8_0_0_1_HAMBI_1287,8,0,0,1,HAMBI_1287
8_0_0_1_HAMBI_1875,8,0,0,1,HAMBI_1875
8_0_0_1_HAMBI_1896,8,0,0,1,HAMBI_1896
8_0_0_1_HAMBI_1972,8,0,0,1,HAMBI_1972
8_0_0_1_HAMBI_1977,8,0,0,1,HAMBI_1977
8_0_0_2_HAMBI_1287,8,0,0,2,HAMBI_1287


,ABNHBPOL_00951,ABNHBPOL_01122,ABNHBPOL_04066,ABNHBPOL_04069,AMIEGGJA_01338,AMIEGGJA_01432,AMIEGGJA_02462,AMIEGGJA_04119,AMIEGGJA_04977,AMIEGGJA_04981,⋯,rhsB_1,rsmG,slmA,spoT_1,tar_2,vgrG1_2,xcpW,xynD_5,ycdT_1,ydcV_4
8_0_0_1_HAMBI_1287,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8_0_0_1_HAMBI_1875,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8_0_0_1_HAMBI_1896,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8_0_0_1_HAMBI_1972,1,1,2,0,0,0,0,0,0,0,⋯,0,0,0,0,0,3,1,0,0,0
8_0_0_1_HAMBI_1977,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8_0_0_2_HAMBI_1287,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [5]:
# T8 for all data

T8 = meta$TIME == 8
meta_T8 = meta[T8,]
data_T8 = data[T8,]
y = colSums(data_T8) > 0
D_T8 = vegdist(data_T8)
M_T8 = adonis(D_T8 ~ IMMIGRATION*ANTIBIOTIC*SPECIES, data = meta_T8)
M_T8


Call:
adonis(formula = D_T8 ~ IMMIGRATION * ANTIBIOTIC * SPECIES, data = meta_T8) 

Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

                                Df SumsOfSqs MeanSqs F.Model      R2 Pr(>F)    
IMMIGRATION                      1     0.122  0.1218   1.724 0.00268  0.112    
ANTIBIOTIC                       1     1.283  1.2834  18.167 0.02825  0.001 ***
SPECIES                          9    37.431  4.1590  58.871 0.82386  0.001 ***
IMMIGRATION:ANTIBIOTIC           1     0.057  0.0565   0.800 0.00124  0.571    
IMMIGRATION:SPECIES              6     0.426  0.0711   1.006 0.00938  0.447    
ANTIBIOTIC:SPECIES               4     0.163  0.0406   0.575 0.00358  0.976    
IMMIGRATION:ANTIBIOTIC:SPECIES   4     0.301  0.0752   1.064 0.00662  0.371    
Residuals                       80     5.652  0.0706         0.12439           
Total                          106    45.434                 1.00000           
---
Signif. codes:  0 ‘***

In [6]:
# T12 for all data

T12 = meta$TIME == 12
meta_T12 = meta[T12,]
data_T12 = data[T12,]
y = colSums(data_T12) > 0
D_T12 = vegdist(data_T12)
M_T12 = adonis(D_T12 ~ IMMIGRATION*ANTIBIOTIC*SPECIES, data = meta_T12)
M_T12


Call:
adonis(formula = D_T12 ~ IMMIGRATION * ANTIBIOTIC * SPECIES,      data = meta_T12) 

Permutation: free
Number of permutations: 999

Terms added sequentially (first to last)

                                Df SumsOfSqs MeanSqs F.Model      R2 Pr(>F)    
IMMIGRATION                      1     0.375  0.3751   4.930 0.00816  0.001 ***
ANTIBIOTIC                       1     0.682  0.6823   8.966 0.01483  0.001 ***
SPECIES                          9    37.468  4.1631  54.708 0.81466  0.001 ***
IMMIGRATION:ANTIBIOTIC           1     0.046  0.0458   0.602 0.00100  0.778    
IMMIGRATION:SPECIES              5     0.565  0.1130   1.485 0.01228  0.027 *  
ANTIBIOTIC:SPECIES               4     0.170  0.0425   0.559 0.00370  0.982    
IMMIGRATION:ANTIBIOTIC:SPECIES   4     0.446  0.1115   1.465 0.00970  0.054 .  
Residuals                       82     6.240  0.0761         0.13567           
Total                          107    45.992                 1.00000           
---
Signif. codes: 

##### Most of the nonsynonymous multi-hit genes occur in the same species across treatments (PERMANOVA species: p < 0.001, R2 > 0.8), likely representing adaptation to experimental conditions. When the presence of species is accounted for, antibiotic also has a significant effect (p < 0.001) on the genes mutated both during and after recovery from the antibiotic pulse. Notably, the effect size of antibiotic decreases when the community recovers (during antibiotic pulse: R2 = 0.028; after recovery: R2 = 0.014). After recovery, immigration also affects the profile of mutated genes for certain species (immigration: p < 0.001; R2 = 0.008; immigration * species: p = 0.041, R2 = 0.012).

# Differentially mutated genes

In [7]:
AB0 = unique(variants$GENE[variants$ANTIBIOTIC == 0])
AB1 = unique(variants$GENE[variants$ANTIBIOTIC != 0])
IM0 = unique(variants$GENE[variants$IMMIGRATION == 0])
IM1 = unique(variants$GENE[variants$IMMIGRATION != 0])
AB0
AB1
IM0
IM1

# Use as input for Venny 2.1 (http://bioinfogp.cnb.csic.es/tools/venny/, accessed 2019-03-25)

[1] vgrG1_2        ltxA           xcpW           ABNHBPOL_00951 ABNHBPOL_01122
 [6] ABNHBPOL_04066 ABNHBPOL_04069 hcpA           slmA           AMIEGGJA_04119
[11] AMIEGGJA_04977 AMIEGGJA_04981 AMIEGGJA_01338 AMIEGGJA_05482 ltrA_2        
[16] AMIEGGJA_01432 AMIEGGJA_05796 AMIEGGJA_05900 AMIEGGJA_05915 rhsB_1        
[21] AMIEGGJA_02462 IBBHIMLJ_00953 cckA           IBBHIMLJ_00246 IBBHIMLJ_00602
[26] IBBHIMLJ_00766 rhaR_1         IBBHIMLJ_00815 IBBHIMLJ_02593 IBBHIMLJ_02849
[31] IBBHIMLJ_02970 IBBHIMLJ_03302 IBBHIMLJ_03495 IBBHIMLJ_03756 IBBHIMLJ_04132
[36] JFDMDMML_01549 LPGKLAMA_00493 LPGKLAMA_00494 LPGKLAMA_00543 alkA_2        
[41] LPGKLAMA_04895 grsB           LPGKLAMA_05062 LPGKLAMA_05116 LPGKLAMA_05119
[46] LPGKLAMA_05251 ydcV_4         potA_5         bca            lgrD_3        
[51] lgrD_4         LPGKLAMA_05584 lgrB           lktA           LPGKLAMA_05944
[56] LPGKLAMA_05943 estA_1         LPGKLAMA_03572 OLKIGDAF_00141 OLKIGDAF_00332
[61] OLKIGDAF_00532 kup            gcvP           dapF           cymR          
[66] gloC_2         OLKIGDAF_00019
91 Levels: ABNHBPOL_00951 ABNHBPOL_01122 ABNHBPOL_04066 ... ydcV_4

[1] cya            vgrG1_2        ltxA           xcpW           ABNHBPOL_00951
 [6] ABNHBPOL_01122 ABNHBPOL_04066 ABNHBPOL_04069 hcpA           phoQ_2        
[11] slmA           CKLFCLNC_00127 eno            tar_2          CKLFCLNC_03364
[16] pta_1          ltxA_6         CKLFCLNC_01286 ycdT_1         DPMDEPGB_02466
[21] DPMDEPGB_03114 spoT_1         DPMDEPGB_03261 kbl            IBBHIMLJ_00953
[26] cckA           IBBHIMLJ_00246 IBBHIMLJ_00389 IBBHIMLJ_00602 IBBHIMLJ_00766
[31] rhaR_1         IBBHIMLJ_00815 IBBHIMLJ_02593 IBBHIMLJ_02849 IBBHIMLJ_02970
[36] IBBHIMLJ_03302 IBBHIMLJ_03495 IBBHIMLJ_03756 IBBHIMLJ_04132 IBBHIMLJ_04193
[41] IDIPBEOP_04409 cspA_6         rsmG           dapF           JFDMDMML_01549
[46] relA_2         JHKIHDGE_01253 xynD_5         JHKIHDGE_03280 LPGKLAMA_00493
[51] LPGKLAMA_00494 LPGKLAMA_00543 alkA_2         LPGKLAMA_04895 grsB          
[56] LPGKLAMA_05062 LPGKLAMA_05116 LPGKLAMA_05119 LPGKLAMA_05251 ydcV_4        
[61] potA_5         bca            lgrD_3         lgrD_4         LPGKLAMA_05584
[66] lgrB           lktA           LPGKLAMA_05944 LPGKLAMA_05943 estA_1        
[71] LPGKLAMA_03572 OLKIGDAF_00141 OLKIGDAF_00332 OLKIGDAF_00532 kup           
[76] gcvP           cymR           gloC_2         OLKIGDAF_00019
91 Levels: ABNHBPOL_00951 ABNHBPOL_01122 ABNHBPOL_04066 ... ydcV_4

[1] cya            vgrG1_2        ltxA           xcpW           ABNHBPOL_00951
 [6] ABNHBPOL_01122 ABNHBPOL_04066 ABNHBPOL_04069 hcpA           phoQ_2        
[11] slmA           AMIEGGJA_04119 AMIEGGJA_04981 AMIEGGJA_01338 ltrA_2        
[16] AMIEGGJA_01432 AMIEGGJA_05796 AMIEGGJA_05900 AMIEGGJA_05915 AMIEGGJA_02462
[21] CKLFCLNC_00127 eno            tar_2          CKLFCLNC_03364 pta_1         
[26] ltxA_6         CKLFCLNC_01286 ycdT_1         DPMDEPGB_02466 DPMDEPGB_03114
[31] spoT_1         DPMDEPGB_03261 kbl            IBBHIMLJ_00953 cckA          
[36] IBBHIMLJ_00246 IBBHIMLJ_00602 IBBHIMLJ_00766 rhaR_1         IBBHIMLJ_00815
[41] IBBHIMLJ_02593 IBBHIMLJ_02849 IBBHIMLJ_02970 IBBHIMLJ_03302 IBBHIMLJ_03495
[46] IBBHIMLJ_03756 IBBHIMLJ_04132 IBBHIMLJ_04193 cspA_6         JFDMDMML_01549
[51] relA_2         JHKIHDGE_01253 xynD_5         JHKIHDGE_03280 LPGKLAMA_00493
[56] LPGKLAMA_00494 LPGKLAMA_00543 alkA_2         LPGKLAMA_04895 grsB          
[61] LPGKLAMA_05062 LPGKLAMA_05116 LPGKLAMA_05119 LPGKLAMA_05251 ydcV_4        
[66] potA_5         bca            lgrD_3         lgrD_4         LPGKLAMA_05584
[71] lgrB           lktA           LPGKLAMA_05944 LPGKLAMA_05943 estA_1        
[76] LPGKLAMA_03572 OLKIGDAF_00141 OLKIGDAF_00332 OLKIGDAF_00532 kup           
[81] gcvP           cymR           gloC_2         OLKIGDAF_00019
91 Levels: ABNHBPOL_00951 ABNHBPOL_01122 ABNHBPOL_04066 ... ydcV_4

[1] vgrG1_2        ltxA           xcpW           ABNHBPOL_00951 ABNHBPOL_01122
 [6] ABNHBPOL_04066 ABNHBPOL_04069 hcpA           phoQ_2         slmA          
[11] AMIEGGJA_04119 AMIEGGJA_04977 AMIEGGJA_04981 AMIEGGJA_01338 AMIEGGJA_05482
[16] ltrA_2         AMIEGGJA_01432 AMIEGGJA_05796 AMIEGGJA_05900 AMIEGGJA_05915
[21] rhsB_1         AMIEGGJA_02462 ycdT_1         DPMDEPGB_03114 spoT_1        
[26] DPMDEPGB_03261 kbl            IBBHIMLJ_00953 cckA           IBBHIMLJ_00246
[31] IBBHIMLJ_00389 IBBHIMLJ_00602 IBBHIMLJ_00766 rhaR_1         IBBHIMLJ_00815
[36] IBBHIMLJ_02593 IBBHIMLJ_02849 IBBHIMLJ_02970 IBBHIMLJ_03302 IBBHIMLJ_03495
[41] IBBHIMLJ_03756 IBBHIMLJ_04132 IBBHIMLJ_04193 IDIPBEOP_04409 cspA_6        
[46] rsmG           cya            dapF           JFDMDMML_01549 relA_2        
[51] JHKIHDGE_01253 xynD_5         LPGKLAMA_00493 LPGKLAMA_00494 LPGKLAMA_00543
[56] alkA_2         LPGKLAMA_04895 grsB           LPGKLAMA_05062 LPGKLAMA_05116
[61] LPGKLAMA_05119 LPGKLAMA_05251 ydcV_4         potA_5         bca           
[66] lgrD_3         lgrD_4         LPGKLAMA_05584 lgrB           lktA          
[71] LPGKLAMA_05944 LPGKLAMA_05943 estA_1         LPGKLAMA_03572 OLKIGDAF_00141
[76] OLKIGDAF_00332 OLKIGDAF_00532 kup            gcvP           cymR          
[81] gloC_2         OLKIGDAF_00019
91 Levels: ABNHBPOL_00951 ABNHBPOL_01122 ABNHBPOL_04066 ... ydcV_4

In [8]:
inters = intersect(AB0, AB1)
inters
AB1_unique = AB1[!(AB1 %in% inters)]
AB1_unique

[1] "vgrG1_2"        "ltxA"           "xcpW"           "ABNHBPOL_00951"
 [5] "ABNHBPOL_01122" "ABNHBPOL_04066" "ABNHBPOL_04069" "hcpA"          
 [9] "slmA"           "IBBHIMLJ_00953" "cckA"           "IBBHIMLJ_00246"
[13] "IBBHIMLJ_00602" "IBBHIMLJ_00766" "rhaR_1"         "IBBHIMLJ_00815"
[17] "IBBHIMLJ_02593" "IBBHIMLJ_02849" "IBBHIMLJ_02970" "IBBHIMLJ_03302"
[21] "IBBHIMLJ_03495" "IBBHIMLJ_03756" "IBBHIMLJ_04132" "JFDMDMML_01549"
[25] "LPGKLAMA_00493" "LPGKLAMA_00494" "LPGKLAMA_00543" "alkA_2"        
[29] "LPGKLAMA_04895" "grsB"           "LPGKLAMA_05062" "LPGKLAMA_05116"
[33] "LPGKLAMA_05119" "LPGKLAMA_05251" "ydcV_4"         "potA_5"        
[37] "bca"            "lgrD_3"         "lgrD_4"         "LPGKLAMA_05584"
[41] "lgrB"           "lktA"           "LPGKLAMA_05944" "LPGKLAMA_05943"
[45] "estA_1"         "LPGKLAMA_03572" "OLKIGDAF_00141" "OLKIGDAF_00332"
[49] "OLKIGDAF_00532" "kup"            "gcvP"           "dapF"          
[53] "cymR"           "gloC_2"         "OLKIGDAF_00019"

[1] cya            phoQ_2         CKLFCLNC_00127 eno            tar_2         
 [6] CKLFCLNC_03364 pta_1          ltxA_6         CKLFCLNC_01286 ycdT_1        
[11] DPMDEPGB_02466 DPMDEPGB_03114 spoT_1         DPMDEPGB_03261 kbl           
[16] IBBHIMLJ_00389 IBBHIMLJ_04193 IDIPBEOP_04409 cspA_6         rsmG          
[21] relA_2         JHKIHDGE_01253 xynD_5         JHKIHDGE_03280
91 Levels: ABNHBPOL_00951 ABNHBPOL_01122 ABNHBPOL_04066 ... ydcV_4

In [9]:
length(unique(variants$GENE))

[1] 91